<a href="https://colab.research.google.com/github/neoxlarge/Liq_Grorilla/blob/main/OpenInterest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
## 顯示 json 物件用 Add this to the first block in your note book
import uuid
from IPython.core.display import display, HTML

import json

def RenderJSON(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)


In [20]:
import webbrowser
def GenerateTable(dataframe,title="data_frame"):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id=title,classes="DataTable")
    return f"""
    <h3>{title}</h3>
    <link href="https://cdn.datatables.net/1.12.1/css/jquery.dataTables.min.css" rel="stylesheet">    
    {table_html}
    <script src="https://code.jquery.com/jquery-3.6.0.slim.min.js" 
            integrity="sha256-u7e5khyithlIdTpu22PHhENmPcRdFiHRjhAuHcs05RI=" crossorigin="anonymous"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.12.1/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                // paging: false,    
                // scrollY: 400,
            }});
        }});
    </script> 
    <br><br>
    """

In [21]:
# 引用lib跟定義數據
import requests
import json
import pandas as pd
import datetime
import time
from sqlalchemy import create_engine #SQL查詢使用
import sqlite3

symbol_list = ['BTC','ETH','SOL','XRP','ADA','DOT','BNB','TRX','LTC','BCH','GMT','AVAX','MATIC','ETC','DOGE','LINK',
         'EOS','SAND','ATOM','NEAR','AXS','APE','FTT','WAVES','FIL','FTM','MANA','SNX','GALA','UNI','CRV','AAVE',
         'UNFI','THETA','XLM','DYDX','ZIL','ALGO','XTZ','XMR','PEOPLE','XEM','BSV','RUNE','COMP','YFI','SUSHI',
         'SHIB','EGLD','1INCH','OP','KNC','ZEC','GRT','RSR','RLC','MKR','STORJ','CHZ','ENJ','OGN','SRM','BIT',
         'CEL','ICP','AR','VET','LUNA2','BAT','NEO','ONE','KAVA','ENS','LRC','GAL','CRO','OMG','BNX','DASH',
         'ALICE','KSM','IMX','ZRX','MTL','FLM','QTUM','IOST','ALT','BTCDOM','LOOKS','IOTA','SXP','HNT',
         'SHIT','FLOW','CELO','LIT','BEL','API3','MASK','ONT','C98','REN','JASMY','BAKE','BAL','ROSE',
         'HBAR','YGG','AUDIO','CHR','RAY','ANKR','ZEN','RVN','SLP','SKL','MID',
         'BAND','CVC','CTK','TRB','BLZ','LPT','KLAY','HOT','STMX','IOTX','ALPHA','PAXG','ANT','ANC','OCEAN',
         'LINA','GTC','CVX','SWRV','ICX','DEFI','COTI','ARPA','MINA','OKB','CELR','SFP','HT','TRU','DENT',
         'REEF','WOO','CTSI','CAKE','ATA','DUSK','GLMR','STX','RNDR','TOMO','DGB','SPELL','FITFI','NKN','DAR',
         'ILV','XAUT','CREAM','SOS','AGLD','BADGER','TLM','BOBA','BICO','JST','UNISWAP','BTS','STEP',
         '1000XEC','TRYB','BRZ','LEO','PERP','TONCOIN','UMA','PRIV','CLV','SC','RON',
         'KDA','RSS3','BSW','BTT','MEDIA','REQ','MAPS','CSPR','STARL',
         'PROM','DODO','1000SHIB'];
symbol_list_fiat = ['USDT', 'USDC']
symbol_list_ftx = ['AMPL','FXS','GST','YFII','BNT','KSHIB','EXCH','ATLAS','FIDA']
symbol_list_okex_only  = ['NYM','WNXM','XCH','GODS']
symbol_list_bybit_only = ['XCN','SHIB1000','ASTR','AKRO']
symbol_list_diff_name  = ['LUNA']
symbol_list_oftenerror = ['BABYDOGE']

debug_list = ['SOL','MATIC','BABYDOGE','TONCOIN','UMA']
cg_headers = {'coinglassSecret': '7ef635a6899a4fbdb9db368d222c974d'}
candles = {"1m": 9, "5m": 3, "15m": 10, "30m": 11, "1h": 2, "4h": 1, "12h": 4, "24h": 5, "90d": 18, "all": 0}

#---------------------------------------------------------------------------------------------
#自定義參數

sqlite_database_file = "oi_database.db"




In [22]:
# 定義資料取得方法
def get_oi_data(symbol, Timeframe="1m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)
    
def get_oi_chart_data(symbol, Timeframe):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest/chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_data(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/longShort_chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance(symbol, timeframe="5m", limit=24):
    url = f"https://www.binance.com/futures/data/globalLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_position(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Position)
    #幣安大戶多空比倉位
    url = f"https://www.binance.com/futures/data/topLongShortPositionRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_account(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Accounts)
    #幣安大戶多空比帳戶
    url = f"https://www.binance.com/futures/data/topLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_liq_detial(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/liquidation/detail/chart?symbol={symbol}&timeType={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_fr(symbol, ttype="U"):
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/funding_rates_chart?symbol={symbol}&type={ttype}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def send_to_bot(tid_, msg, bot_token_):
    url = f'https://api.telegram.org/bot{bot_token_}/sendMessage'
    headers = {'content-type': 'application/json'}
    data  = f'{{"chat_id": "{tid_}", "text": "{msg}", "parse_mode": "HTML", "disable_web_page_preview": true}}'.encode('utf-8')
    r = requests.post(url, data=data, headers=headers, verify=False)


In [23]:
#daytime 統一daytime格式
import datetime as dt
dt_now = int(dt.datetime.now().timestamp()) * 1000                          #現在時間timestamp, UTC0
dt_now_utc8 = dt_now + 8 * 60 * 60 * 1000                            #現在時間timestamp, UTC+8
dt_last1hr = int(dt.datetime.now().timestamp())  // 3600 * 3600 * 1000              #最近一個整數小時
dt_72hr_ago = (int(dt.datetime.now().timestamp()) // 3600 * 3600 - 72 * 60 * 60) * 1000   #72個小時前

print(f"{dt_now}")
print(f"{dt_now_utc8}")
print(f"{dt_last1hr}")
print(f"{dt_72hr_ago}")

1660616934000
1660645734000
1660615200000
1660356000000


In [24]:
# 建立幣總表 2百多筆要約2分多鐘
# 定義欄位
col = ['symbol',"Price","OI_1h","OI_4h","OI_24h","OI$M","FR"]
df = pd.DataFrame(columns=col) # df 是用pandas建立的表格
error_msg = ""
for symbol in symbol_list: # 每一個幣的資料
    try:
        coin_oi_data = get_oi_data(symbol)
        oi_1h  = coin_oi_data["data"][0]["h1OIChangePercent"]
        oi_4h  = coin_oi_data["data"][0]["h4OIChangePercent"]
        oi_24h = coin_oi_data["data"][0]["h24Change"]
        fr     = coin_oi_data["data"][0]["avgFundingRate"]
        oi_mil = coin_oi_data["data"][0]["openInterest"]/1000000
        price  = coin_oi_data["data"][1]["price"]

        # 新增一筆資料至建立的表格df 
        df.loc[df.shape[0]] = [symbol,f"${price:0.4f}",oi_1h,oi_4h,oi_24h,f"${oi_mil:0.2f}M",f"{fr:0.4f}%"]
    
        time.sleep(200/1000) # api太頻繁取coinglass會擋
    except Exception as e:
        error_msg = error_msg + type(e).__name__ + " " + str(e).replace('"', "'")
        print(symbol + " " + error_msg)

# 增加 LSUR 欄位
col.append("LSUR")
df = df.reindex(columns=col) 

# 總表以 OI_1h 升序取前10筆
df_1h_up10   = df.sort_values(by="OI_1h", ascending=False).head(10)  
# 10筆表以 symbol 當 index
df_1h_up10.set_index('symbol', inplace = True)
# 試印 df_1h_up10
print(df_1h_up10) 

SWRV KeyError 'data'
          Price  OI_1h  OI_4h  OI_24h      OI$M        FR  LSUR
symbol                                                         
CHZ     $0.2080  14.08   0.62   66.02  $119.36M   0.0076%   NaN
XEM     $0.0529   1.75   1.90   12.09   $12.36M   0.0098%   NaN
THETA   $1.4934   1.38   1.19    0.28   $61.64M   0.0085%   NaN
RAY     $0.8410   1.35   1.52   -1.83   $10.05M  -0.0000%   NaN
BIT     $0.5200   1.31  -5.51  -13.31   $21.53M   0.0051%   NaN
STMX    $0.0099   1.23   0.09  -10.98    $5.92M   0.0126%   NaN
SPELL   $0.0017   1.17  -7.24   27.73    $3.70M  -0.0037%   NaN
MATIC   $0.9580   0.94  -2.57  -17.90  $266.28M  -0.0025%   NaN
SHIB    $0.0000   0.88  -1.16  -28.78   $40.96M   0.0028%   NaN
GMT     $1.0663   0.83  -0.18    3.13  $182.16M  -0.0117%   NaN


In [25]:
# 產生網頁js表格
html = GenerateTable(df)
# write the HTML content to an HTML file
open("index.html", "w").write(html)
# open the new HTML file with the default browser
webbrowser.open("index.html")
        

False

In [26]:
coin_lsur = get_lsur_data("ETC", "5m")
RenderJSON(coin_lsur)
# coin_lsur["data"]["longShortRateList"][-1]

In [27]:
lsur3 = get_lsur_binance("nkn", "1h", 8)
RenderJSON(lsur3)

In [28]:
# 建立全部幣的OI,LSUR,FR列表, 2百多筆要約8分多鐘
# 命名格式
# 1.API取回: _data
# 2.Pandas dataframe: _df
# 3.Sqlite3: _table, _view
# 定義欄位
col = ["symbol","TOP1_EX","Price_1h","Price_8h","OI_1h_CG","OI_8h_CG",
     "OI_1h_BN","OI_8h_BN","LSUR_1h_BN","LSUR_8h_BN","LSUR_1h_BN_TOP_A",
     "LSUR_8h_BN_TOP_A","LSUR_1h_BN_TOP_P","LSUR_8h_BN_TOP_P",
     "LSUR_1h_CG","LSUR_8h_CG","FR","Timestamp"]
error_msg = ""
oi_df = pd.DataFrame(columns=col) # df 是用pandas建立的表格

for symbol in debug_list : # 每一個幣的資料
    try:
        coin_oi_data = get_oi_chart_data(symbol,"1h")

        #Top1持倉交易所
        top1_exchanger = [(x,coin_oi_data["data"]["dataMap"][x][-1]) for x in coin_oi_data["data"]["dataMap"]]
        top1_exchanger.sort(key = lambda y : y[1],reverse=True)
        top1_exchanger = top1_exchanger[0][0]

        #Binance 單一交易所持倉        
        if "Binance" in coin_oi_data["data"]["dataMap"].keys():
            oi_1h_BN = float(coin_oi_data["data"]["dataMap"]["Binance"][-1])
            oi_8h_BN = float(coin_oi_data["data"]["dataMap"]["Binance"][-8])
        else:
            oi_1h_BN = float(0)
            oi_8h_BN = float(0)    
        
        #Coinglass 全部交易所持倉,自行加總

        def none_to_0(input_): #部分數值為None, 可能會出錯, 把None改成0
            if input_ == None:
                return 0
            else: return input_
               
        oi_1h_CG = float(0)
        for i in coin_oi_data["data"]["dataMap"].keys():
            oi_1h_CG = oi_1h_CG + none_to_0(coin_oi_data["data"]["dataMap"][i][-1])
                    
        oi_8h_CG = float(0)
        for j in coin_oi_data["data"]["dataMap"].keys():
            oi_8h_CG = oi_8h_CG + none_to_0(coin_oi_data["data"]["dataMap"][j][-8])
        
        #幣價              
        price_1h  = float(coin_oi_data["data"]["priceList"][-1])
        price_8h  = float(coin_oi_data["data"]["priceList"][-8])
        
        #幣安多空比
        lsur_data_BN = get_lsur_binance(symbol, "1h", 8)

        if lsur_data_BN:
          lsur_1h_BN = float(lsur_data_BN[-1]["longShortRatio"])
          lsur_8h_BN = float(lsur_data_BN[0]["longShortRatio"])                
        else:
          lsur_1h_BN = float(0)
          lsur_8h_BN = float(0)

        #幣安大戶多空比帳號
        lsur_data_BN_top_account = get_lsur_binance_top_account(symbol, "1h", 8) 
        
        if lsur_data_BN_top_account:
          lsur_1h_BN_top_ac = float(lsur_data_BN_top_account[-1]["longShortRatio"])
          lsur_8h_BN_top_ac = float(lsur_data_BN_top_account[0]["longShortRatio"])                
        else:
          lsur_1h_BN_top_ac = float(0)
          lsur_8h_BN_top_ac = float(0)

        #幣安大戶多空比倉位
        lsur_data_BN_top_position = get_lsur_binance_top_position (symbol, "1h", 8) 
        
        if lsur_data_BN_top_position:
          lsur_1h_BN_top_po = float(lsur_data_BN_top_position[-1]["longShortRatio"])
          lsur_8h_BN_top_po = float(lsur_data_BN_top_position[0]["longShortRatio"])                
        else:
          lsur_1h_BN_top_po = float(0)
          lsur_8h_BN_top_po = float(0)

        #Coinglass多空比
        lsur_data_CG = get_lsur_data(symbol,"1h")

        if lsur_data_CG:
          lsur_1h_CG = float(lsur_data_CG["data"]["longShortRateList"][-1])
          lsur_8h_CG = float(lsur_data_CG["data"]["longShortRateList"][-8])                
        else:
          lsur_1h_CG = float(0)
          lsur_8h_CG = float(0)
           
        #資費
        fr_data = get_fr(symbol)
          #fr先抓幣安的,如果沒幣安的,就先填0吧,以後再改
          #修改成抓top1交易所的fr
        if top1_exchanger in fr_data["data"]["dataMap"].keys():
          fr_1h = fr_data["data"]["dataMap"][top1_exchanger][-1]
        else:
          fr_1h = 0
        #timestamp,計算次數會用到
        timestamp = coin_oi_data["data"]["dateList"][-1]
        
        # 新增一筆資料至建立的表格df 
        oi_df.loc[oi_df.shape[0]] = [symbol,top1_exchanger,
                        price_1h,price_8h,oi_1h_CG,oi_8h_CG,
                        oi_1h_BN,oi_8h_BN,lsur_1h_BN,lsur_8h_BN,
                        lsur_1h_BN_top_ac,lsur_8h_BN_top_ac,
                        lsur_1h_BN_top_po,lsur_8h_BN_top_po,
                        lsur_1h_CG,lsur_8h_CG,fr_1h,timestamp]
          
        time.sleep(200/1000) # api太頻繁取coinglass會擋
    except Exception as e:
        error_msg = error_msg + type(e).__name__ + " " + str(e).replace('"', "'")
        print(symbol + " " + error_msg)

# 增加 LSUR 欄位
#col.append("LSUR")
#df = df.reindex(columns=col) 
#print(oi_df.head(5))

BABYDOGE TypeError float() argument must be a string or a number, not 'NoneType'


In [29]:
#sqlite 資料庫操作
#1.將pandas全部幣的資料表格存入SQL,以利用SQL查詢
#2.Query出數據流訊號表: oi_view_BN, oi_view_BN
#3.將數據流訊號表存當成oi_table_msgcount表,計算72小時出現的次數



#應該可以復刻班的數據流訊號
#Binance oi change > 20%
#Binance LSUR change < -10%
sql_create_oi_view_BN = """
create temp view oi_view_BN as
select 
  symbol,
  TOP1_EX,
  Price_1h,
  Price_8h,
  (Price_1h - Price_8h) / Price_8h as Price_change,
  OI_1h_BN as OI_1h,
  OI_8h_BN as OI_8h,
  ((OI_1h_BN - OI_8h_BN) / OI_8h_BN) as OI_change,
  LSUR_1h_BN as LSUR_1h,
  LSUR_8h_BN as LSUR_8h, 
  ((LSUR_1h_BN - LSUR_8h_BN)/LSUR_8h_BN) as LSUR_change, 
  FR,
  Timestamp
from oi_table_all 
where 
  OI_change > 0.2 
  and LSUR_change < -0.1
order by OI_change desc"""
#以Coinglass的全網持倉和全網多空比抓資料,排除掉幣安的資料.
#OI_change > 0.2 and LSUR_change < -0.1 and OI_1h_BN < 1
sql_create_oi_view_CG = """
create temp view oi_view_CG as
select 
  symbol,
  TOP1_EX,
  Price_1h,
  Price_8h,
  (Price_1h - Price_8h) / Price_8h as Price_change,
  OI_1h_CG as OI_1h,
  OI_8h_CG as OI_8h,
  ((OI_1h_CG - OI_8h_CG) / OI_8h_CG) as OI_change,
  LSUR_1h_CG as LSUR_1h,
  LSUR_8h_CG as LSUR_8h, 
  ((LSUR_1h_CG - LSUR_8h_CG)/LSUR_8h_CG) as LSUR_change, 
  FR,
  Timestamp
from oi_table_all 
where 
  OI_change > 0.2 
  and LSUR_change < -0.1
  and OI_1h_BN < 1
order by OI_change desc
"""


#建立oi_table_msgcount表格
sql_create_oi_table_msgcount = """
create table if not exists oi_table_msgcount (
symbol text,
TOP1_EX text,
Price_1h real,
Price_8h real,
Price_change real,
OI_1h real,
OI_8h real,
OI_change real,
LSUR_1h real,
LSUR_8h real,
LSUR_change real,
FR real,
Timestamp integer
);
"""


#sql script
#1.oi_view_BN, oi_view_BN, 加入到oi_table_msgcount表格

sql_merge_to_oi_table_msgcount_script = """
insert into 
  oi_table_msgcount(symbol,TOP1_EX,Price_1h,Price_8h,Price_change,OI_1h,OI_8h,OI_change,LSUR_1h,LSUR_8h,LSUR_change,FR,Timestamp)
select
  symbol,TOP1_EX,Price_1h,Price_8h,Price_change,OI_1h,OI_8h,OI_change,LSUR_1h,LSUR_8h,LSUR_change,FR,Timestamp
from
  oi_view_BN;   

insert into 
  oi_table_msgcount(symbol,TOP1_EX,Price_1h,Price_8h,Price_change,OI_1h,OI_8h,OI_change,LSUR_1h,LSUR_8h,LSUR_change,FR,Timestamp)
select
  symbol,TOP1_EX,Price_1h,Price_8h,Price_change,OI_1h,OI_8h,OI_change,LSUR_1h,LSUR_8h,LSUR_change,FR,Timestamp
from
  oi_view_CG;

"""

sql_create_oi_view_LSURlessthan1 = """
create temp view oi_view_lsurlessthan1 as
select 
  symbol,
  TOP1_EX,
  Price_1h,
  Price_8h,
  (Price_1h - Price_8h) / Price_8h as Price_change,
  OI_1h_BN as OI_1h,
  OI_8h_BN as OI_8h,
  ((OI_1h_BN - OI_8h_BN) / OI_8h_BN) as OI_change,
  LSUR_1h_BN as LSUR_1h,
  LSUR_8h_BN as LSUR_8h, 
  ((LSUR_1h_BN - LSUR_8h_BN)/LSUR_8h_BN) as LSUR_change, 
  FR,
  Timestamp
from oi_table_all 
where 
  (LSUR_1h_BN < 1 and LSUR_1h_BN > 0)
  and (LSUR_1h_BN_TOP_A < 1 and LSUR_1h_BN_TOP_A > 0)
  and (LSUR_1h_CG < 1 and LSUR_1h_CG > 0)
order by OI_change desc"""



#建立sqlite3本地儲存資料庫檔案
conn = sqlite3.connect(sqlite_database_file)
cur = conn.cursor()

#將oi_df全幣表轉到sqlite oi_table_all
cur.execute("drop table if exists oi_table_all")
conn.commit()
oi_df.to_sql('oi_table_all', conn)

#create sqlite view oi_view_BN oi_view_BN
cur.execute(sql_create_oi_view_BN)
cur.execute(sql_create_oi_view_CG)
cur.execute(sql_create_oi_view_LSURlessthan1)

#讀取到dataframe
oi_df_BN = pd.read_sql("select * from oi_view_BN",conn)
oi_df_CG = pd.read_sql("select * from oi_view_CG",conn)
oi_df_lsurlessthan1 = pd.read_sql("select * from oi_view_lsurlessthan1",conn)

#oi_view_BN, oi_view_CG匯整到oi_table_msgcount表中
cur.execute(sql_create_oi_table_msgcount) #如沒有table會新增table
cur.executescript(sql_merge_to_oi_table_msgcount_script)

conn.close()

print(oi_df_BN)

Empty DataFrame
Columns: [symbol, TOP1_EX, Price_1h, Price_8h, Price_change, OI_1h, OI_8h, OI_change, LSUR_1h, LSUR_8h, LSUR_change, FR, Timestamp]
Index: []


In [30]:

def generate_cglink(symbol, exchange):
   # 交易所連結格式
   #BINANCE https://www.coinglass.com/zh-TW/tv/Binance_BTCUSDT
   #BYBIT https://www.coinglass.com/zh-TW/tv/Bybit_BTCUSDT
   #FTX  https://www.coinglass.com/zh-TW/tv/FTX_GMT-PERP
   #OKEX https://www.coinglass.com/zh-TW/tv/Okex_BTC-USD-SWAP
   #Bitget https://www.coinglass.com/zh-TW/tv/Bitget_ETHUSDT_UMCBL
   #Huobi https://www.coinglass.com/zh-TW/tv/Huobi_WOO-USDT
   
   if exchange in ["Binance","Bybit"]:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "_" + symbol + "USDT"
   elif exchange in ["FTX"]:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "_" + symbol + "-PERP"
   elif exchange in ["Okex"]:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "-" + symbol + "-USD-SWAP"
   elif exchange in ["Bitget"]:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "_" + symbol + "USDT_UMCBL"
   elif exchange in ["Huobi"]:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "_" + symbol + "-USDT"
   else:
     data = "https://www.coinglass.com/zh-TW/tv/" + exchange + "_" + symbol + "USDT"        

   data_link = "<a href=\\\"" + data + "\\\">" + exchange + "_" + symbol + "</a>"
   #返回一個hyperlink
   return data_link

def df_to_tg_bot(df,title="test",bot_chat_id="1011587295"):
#Coinglass OI channel chat_id:-1001647705274
#test bot chat_id: 1011587295
    #沒資料時不送訊息
    if not df.empty:    
        
        msg = []
        msg_link = []
        #持倉資料最後時間, UTC+8 
        formatted_time = dt.datetime.fromtimestamp(((df.at[0,'Timestamp']+8*60*60*1000)/1000)//3600*3600).strftime('%m-%d %H:%M')
        
        #msg line1
        dt_now_filename = f"{dt_now}.html"
        html_file = f"/datasets/coinglassxoixlsur/web/{dt_now_filename}"
        web_link = f"https://fefk3tqrazbqkxf6i6kz4q.on.drv.tw/web/{dt_now_filename}"
        msg.append(f"<a href='{web_link}'>{title}</a> Date:{formatted_time}\n")
        
        #msg line2 ,先列出幣, 以利通知訊息可以先看到幣名
        symbol_list=[ f"#{x}" for x in df['symbol']]
        symbol_list = " ".join(symbol_list)  
        msg.append(f"{symbol_list} \n")
        msg.append('<code>Coin</code>  Price(change) <code>OI(change)</code> LSUR(change) <code>FR</code> \n')

        #開資料庫,準備查次數
        conn = sqlite3.connect(sqlite_database_file)
        cur = conn.cursor()
            
        #msg line3,　讀取oi_df_BN表格內容
        msg.append(f"---------------------------------------------------------------- \n")
        for ix in df["symbol"].keys():
            msg_sy = f"{df.at[ix,'symbol']}"
            msg_price = f"{df.at[ix,'Price_1h']:5.4f}({df.at[ix,'Price_change']:.0%})"
            msg_oi = f"{df.at[ix,'OI_1h']/1000000:.2f}M({df.at[ix,'OI_change']:.0%})"
            msg_lsur = f"{df.at[ix,'LSUR_1h']:.2f}({df.at[ix,'LSUR_change']:.0%})"
            msg_fr = f"{df.at[ix,'FR']:.4f}"
            msg_col = f"<code>#{msg_sy:<5}</code>{msg_price: <13}<code>{msg_oi: <12}</code>{msg_lsur: <12}<code>{msg_fr}</code>\n" 

            msg.append(msg_col)

            #產生連結,存到list裡,訊息尾使用
            link = generate_cglink(df.at[ix,'symbol'],df.at[ix,'TOP1_EX'])
            
            sql_msgCount = f"select count(*) as msg_count from oi_table_msgcount where symbol ='{df.at[ix,'symbol']}' and timestamp > {dt_72hr_ago} group by symbol"
            cur.execute(sql_msgCount)
            msgcount = cur.fetchall()
                    
            if msgcount != [] and msgcount[0][0] > 1:
              link = f"{link} :  {msgcount[0][0]} times in 72hrs \n"
            else: 
              link = f"{link}\n"
            msg_link.append(link)

        conn.close()
        #msg line4, Top1 exchanger and hyperlink
        msg.append('---------------------------------------------------------------- \n Top1 Exchanger(link):  \n')
        for i in msg_link:
            msg.append(i)

        #Coinglass OI channel chat_id:-1001647705274
        #test bot chat_id: 1011587295
        msg_all =""
        for i in msg:
            msg_all += i
        
        bot_token = '5556315407:AAHpetcMV8PSuW2xWZo012mvTsRuKvqVPMY'
        #chat_id = '1011587295'
        send_to_bot(bot_chat_id,msg_all,bot_token)    
        return msg        
        



In [31]:
#執行step
#setp1:建立sql查詢
#sql查詢結果存到dataframe
#將dataframe傳給df_to_tg_bot.

df_to_tg_bot(oi_df_BN,"test🐮🐮Binance OI")
df_to_tg_bot(oi_df_CG,"test🐮🐮Coinglass OI")
df_to_tg_bot(oi_df_lsurlessthan1,"test💵💵幣安,全網多空比,大戶多空(帳戶)小於1")

In [32]:
#Liquidation 爆倉流 建立dataframe

#自定coin
watchinglist_symbol = {'AVAX','MATIC'} #set可消附重復的值.再轉回list即可

conn = sqlite3.connect(sqlite_database_file)
cur = conn.cursor()
sql_symbolin72hr = f"select symbol from oi_table_msgcount where timestamp > {dt_72hr_ago} group by symbol"
coin = cur.execute(sql_symbolin72hr)
coin2 = coin.fetchall()
for i in coin2:
    watchinglist_symbol.add(i[0]) 
watchinglist_symbol = list(watchinglist_symbol)

col =["symbol","top1_ex","timestamp","price_1h"] + [f"liq_{i}h_sell" for i in range(1,13)] + [f"liq_{i}h_buy" for i in range(1,13)]

liq_df = pd.DataFrame(columns=col) 
error_msg = ""
for i in watchinglist_symbol:
    try:
        liq_data = get_liq_detial(i,"1h")
        liq_data = liq_data["data"]
        time.sleep(200/1000)
        liq_1h_list_buy = []
        liq_1h_list_sell = []
        shift_t = -1
        while liq_data[shift_t]["createTime"] >= dt_last1hr:
            shift_t -= 1

        for j in liq_data[shift_t:(shift_t-12):-1]:
            liq_1h_list_buy.append(j["buyVolUsd"])
            liq_1h_list_sell.append(j["sellVolUsd"])

        #找出爆倉最多的交易所
        liq_data_top1_ex = [(x["exchangeName"],x["sellVolUsd"]-x["buyVolUsd"]) for x in liq_data[shift_t]["list"]]
        liq_data_top1_ex.sort(key = lambda y : y[1],reverse=True)    
        liq_data_top1_ex = liq_data_top1_ex[0][0]  

        liq_1h_list_sellbuy = [i,liq_data_top1_ex,liq_data[shift_t]["createTime"],liq_data[shift_t]["price"]] + liq_1h_list_sell + liq_1h_list_buy
    
        liq_df.loc[liq_df.shape[0]] = liq_1h_list_sellbuy
    except Exception as e:
        error_msg = error_msg + type(e).__name__ + " " + str(e).replace('"', "'")
        print(i + " " + error_msg)

conn.close()
print(liq_df)


  symbol top1_ex      timestamp  price_1h  liq_1h_sell   liq_2h_sell  \
0   AVAX    Okex  1660611600000   27.8500          0.0  22692.982095   
1  MATIC     FTX  1660611600000    0.9538          0.0   4423.475917   

     liq_3h_sell  liq_4h_sell   liq_5h_sell  liq_6h_sell  ...    liq_3h_buy  \
0  143797.510250  36188.88110             0      0.00000  ...  66275.267200   
1  204887.130762  25727.67939  13368.148827   1891.64233  ...  34379.208775   

   liq_4h_buy  liq_5h_buy  liq_6h_buy  liq_7h_buy liq_8h_buy  liq_9h_buy  \
0       0.000           0     6201.63         0.0   0.000000      0.0000   
1   22623.543         0.0        0.00         0.0  13.346655    252.7002   

   liq_10h_buy  liq_11h_buy  liq_12h_buy  
0   1209.77400   2881.55650            0  
1  54302.82874  10069.10526       8649.9  

[2 rows x 28 columns]


In [33]:
print(liq_df)
def format_dataframe(df,col):
    for i in df:
        print(i)


  symbol top1_ex      timestamp  price_1h  liq_1h_sell   liq_2h_sell  \
0   AVAX    Okex  1660611600000   27.8500          0.0  22692.982095   
1  MATIC     FTX  1660611600000    0.9538          0.0   4423.475917   

     liq_3h_sell  liq_4h_sell   liq_5h_sell  liq_6h_sell  ...    liq_3h_buy  \
0  143797.510250  36188.88110             0      0.00000  ...  66275.267200   
1  204887.130762  25727.67939  13368.148827   1891.64233  ...  34379.208775   

   liq_4h_buy  liq_5h_buy  liq_6h_buy  liq_7h_buy liq_8h_buy  liq_9h_buy  \
0       0.000           0     6201.63         0.0   0.000000      0.0000   
1   22623.543         0.0        0.00         0.0  13.346655    252.7002   

   liq_10h_buy  liq_11h_buy  liq_12h_buy  
0   1209.77400   2881.55650            0  
1  54302.82874  10069.10526       8649.9  

[2 rows x 28 columns]


In [34]:
#Liquidation 爆倉流 dataframe to sqlite

#建立sqlite3本地儲存資料庫檔案
conn = sqlite3.connect(sqlite_database_file)
cur = conn.cursor()

#將oi_df全幣表轉到sqlite oi_table_all
cur.execute("drop table if exists liq_table")
conn.commit()
liq_df.to_sql("liq_table", conn)

#淨多單(多單-空單))爆倉大於100k
sql_create_liq_view_sell_1h = """
create temp view liq_view_sell_1h as
select 
    symbol,
    top1_ex,
    timestamp,
    price_1h,
    liq_1h_sell,
    liq_1h_buy,
    (liq_1h_sell+liq_2h_sell+liq_3h_sell+liq_4h_sell+liq_5h_sell) - (liq_1h_buy+liq_2h_buy+liq_3h_buy+liq_4h_buy+liq_5h_buy) as liq_5h_sum 
from 
    liq_table 
where 
    (liq_1h_sell - liq_1h_buy) > 10 
order by
    liq_1h_sell desc
"""
#5小時內淨多單大於300K
sql_create_liq_view_sell_5h = """
create temp view liq_view_sell_5h as
select 
    symbol,
    (liq_1h_sell - liq_1h_buy) as liq_1h_sell,
    liq_1h_sell,
    (liq_1h_sell+liq_2h_sell+liq_3h_sell+liq_4h_sell+liq_5h_sell) as liq_5h_sum 
from 
    liq_table 
where 
    (liq_1h_sell+liq_2h_sell+liq_3h_sell+liq_4h_sell+liq_5h_sell) - (liq_1h_buy+liq_2h_buy+liq_3h_buy+liq_4h_buy+liq_5h_buy) > 300000
"""

sql_test = """


"""

cur.execute(sql_create_liq_view_sell_1h)
liq_df_sell_1h = pd.read_sql("select * from liq_view_sell_1h", conn)


conn.close()
print(liq_df_sell_1h)

Empty DataFrame
Columns: [symbol, top1_ex, timestamp, price_1h, liq_1h_sell, liq_1h_buy, liq_5h_sum]
Index: []


In [35]:
def df_to_tg_liq_bot(df,title="test",bot_chat_id="1011587295"):
#Coinglass OI channel chat_id:-1001647705274
#test bot chat_id: 1011587295
    
    #沒資料時不送訊息
    if not df.empty:

        conn = sqlite3.connect(sqlite_database_file)
        cur = conn.cursor()

        msg = []
        msg_link = []
        
        #持倉資料最後時間, UTC+8 
        formatted_time = dt.datetime.fromtimestamp(((df.at[0,'timestamp']+8*60*60*1000) / 1000)//3600 * 3600).strftime('%m-%d %H:%M')
        
        #msg line1
        dt_now_filename = f"{dt_now}.html"
        html_file = f"/datasets/coinglassxoixlsur/web/{dt_now_filename}"
        web_link = f"https://fefk3tqrazbqkxf6i6kz4q.on.drv.tw/web/{dt_now_filename}"
        msg.append(f"<a href='{web_link}'>{title}</a> Date:{formatted_time}\n")
        
        #msg line2 ,先列出幣, 以利通知訊息可以先看到幣名
        symbol_list=[ f"#{x}" for x in df['symbol']]
        symbol_list = " ".join(symbol_list)
        msg.append(f"{symbol_list} \n")

        #symbol, price ,淨多單爆, 空單爆, 5小時多單共爆
        msg.append('<code>Coin</code> 價格  <code>多單爆</code> 空單爆 <code>5h淨多單共爆</code>\n')
            
        #msg line3,讀取liq_sell_1h表格內容
        msg.append(f"-------------------------------- \n")
        for i in df["symbol"].keys():
            msg_symbol = f"{df.at[i,'symbol']}"
            msg_price = f"{df.at[i,'price_1h']:.3f}"
            msg_liq_1h_sell = f"{df.at[i,'liq_1h_sell']/1000:.2f}K"
            msg_liq_1h_buy = f"{df.at[i,'liq_1h_buy']/1000:.2f}K"
            msg_liq_5h_sum = f"{df.at[i,'liq_5h_sum']/1000:.2f}K"
            msg_col = f"<code>#{msg_symbol:<5} </code>{msg_price}  <code> {msg_liq_1h_sell}</code>  {msg_liq_1h_buy} <code> {msg_liq_5h_sum}</code> \n" 
            print(msg_liq_1h_sell)
            msg.append(msg_col)

            msg_link.append(f"{generate_cglink(df.at[i,'symbol'],df.at[i,'top1_ex'])} \n")
            
        #msg line4, Top1 exchanger and hyperlink
        msg.append('--------------------------------\n')
        for i in msg_link:
            msg.append(i)

        #Coinglass OI channel chat_id:-1001647705274
        #test bot chat_id: 1011587295
        msg_all =""
        for i in msg:
            msg_all += i
        
        bot_token = '5556315407:AAHpetcMV8PSuW2xWZo012mvTsRuKvqVPMY'
        #chat_id = '1011587295'
        send_to_bot(bot_chat_id,msg_all,bot_token)    
        return msg        
        
        conn.close()


In [36]:
print(liq_df_sell_1h)
df_to_tg_liq_bot(liq_df_sell_1h,"test💥💥Liquidation alert")


Empty DataFrame
Columns: [symbol, top1_ex, timestamp, price_1h, liq_1h_sell, liq_1h_buy, liq_5h_sum]
Index: []


In [37]:
#寫到html
import os

dt_now_filename = f"{dt_now}.html"

curr_path = os.getcwd()
if curr_path == '/content':  #colab path
    html_file = f"/content/{dt_now_filename}" 
elif curr_path == '/work': #deepnote path
    html_file = f"/datasets/coinglassxoixlsur/web/{dt_now_filename}"

web_link = f"https://fefk3tqrazbqkxf6i6kz4q.on.drv.tw/web/{dt_now_filename}"    



# 產生網頁js表格
show_time = f"<h3>{dt.datetime.fromtimestamp(dt_now_utc8 /1000).strftime('%Y-%m-%d %H:%M')}</h3>"
oi_BN_html = GenerateTable(oi_df_BN,"Binance_OI")
oi_CG_html = GenerateTable(oi_df_CG,"Conglass_OI")
oi_lsurlessthan1_thml = GenerateTable(oi_df_lsurlessthan1,"LSUR_all_less_than_1")
liq_html = GenerateTable(liq_df_sell_1h,"Liquidation_last1_hour")
html = show_time + oi_BN_html + oi_CG_html + oi_lsurlessthan1_thml + liq_html
# write the HTML content to an HTML file
open(html_file, "w").write(html)



4436